In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [58]:
bnf_pandas = pd.read_csv("../data/bnf_mtrend_data.csv")

# INDEX = 'MIDCP'
INDEX = 'BNF'
# INDEX = 'NIFTY'
# INDEX = 'FNF'
# INDEX = 'STOCK'
# INDEX = 'SENSEX'

if INDEX == 'BNF':
    lev_ = 5.066
    slippage_ = 0.0001
    lot_size_ = 15
elif INDEX == 'NIFTY' or INDEX == 'FNF':
    lev_ = 6.333
    slippage_ = 0.0002
    lot_size_ = 25
elif INDEX == 'MIDCP':
    lev_ = 6.333
    slippage_ = 0.0005
    lot_size_ = 50
elif INDEX == 'SENSEX':
    lev_ = 7
    slippage_ = 0.0001
    lot_size_ = 10
elif INDEX == 'STOCK':
    lev_ = 5
    slippage_ = 0.001
    lot_size_ = 1
# bnf_pandas = pd.read_csv("../data/finnifty_1hr_tv (2).csv")
# bnf_pandas = pd.read_csv('../data/midcp_1hr_tv (4).csv')
# bnf_pandas = pd.read_csv('../data/finnifty_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/bnf_fut_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')


In [59]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2015]
bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [60]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

<class 'polars.dataframe.frame.DataFrame'>


In [61]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


In [62]:
# Parameters
ema_short_period = 9
ema_long_period = 21
rsi_period = 14
rsi_short_threshold = 50
rsi_oversold = 30
# stop_loss_pct = 0.5 / 100  # 0.5%
# take_profit_pct = 1.0 / 100  # 1.0%

In [63]:
df = bnf_pandas

In [64]:
# df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
# df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

In [65]:
# Calculate RSI
# delta = df["close"].diff()
# gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
# loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
# rs = gain / loss
# df["rsi"] = 100 - (100 / (1 + rs))

In [66]:
def short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold):

    df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
    df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

    # Calculate RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
    rs = gain / loss
    df["rsi"] = 100 - (100 / (1 + rs))
    
    # Initialize variables
    position = None  # None, 'short'
    tradebook = []
    signal_time = None
    signal_price = None
    entry_time = None
    entry_price = None
    initial_sl = None
    
    # Iterate through rows to generate signals
    for i in range(len(df)):
        row = df.iloc[i]
    
        # Check for short signal condition
        if position is None:
            if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                if signal_price is None:  # New signal
                    signal_time = row["datetime"]
                    signal_price = row["low"]
                    # print(f'New Signal : {signal_time} , {signal_price}')
                else:
                    if row['low'] >= signal_price:
                        # Update Signal Price to Better Candle low
                        signal_time = row["datetime"]
                        signal_price = row["low"]
    
            if signal_price is not None:
                if row["low"] < signal_price and row['high'] >= signal_price:  # Signal broken
                    entry_time = row["datetime"]
                    entry_price = signal_price
                    initial_sl = max(df["high"][max(0, i - 3):i].max(), row["high"])
                    position = "short"
        
                    # Add trade to tradebook
                    tradebook.append({
                        "Signal Time": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": None,
                        "Exit Price": None,
                        "Points Captured": None
                    })
                    signal_time, signal_price = None, None  # Reset signal
                else:
                    signal_price = None
                    if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                        if signal_price is None:  # New signal
                            signal_time = row["datetime"]
                            signal_price = row["low"]
                        else:
                            if row['low'] >= signal_price:
                                # Update Signal Price to Better Candle low
                                signal_time = row["datetime"]
                                signal_price = row["low"]
    
        # Exit logic for short position
        if position is not None:
            if row['close'] >= initial_sl:
                exit_time = row["datetime"]
                exit_price = row['close']
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'Initial SL Hit'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
            elif row["rsi"] > rsi_oversold:
                exit_time = row["datetime"]
                exit_price = row["close"]
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'RSI Oversold'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
    
    # Convert tradebook to a DataFrame
    tradebook_df = pd.DataFrame(tradebook)
    return tradebook_df
    # Output
    # print(tradebook_df)
    
    # Optional: Save to CSV
    # tradebook_df.to_csv("tradebook.csv", index=False)


In [67]:
ema_short_period = 5
ema_long_period = 7
rsi_period = 6
rsi_short_threshold = 60
rsi_oversold = 50
tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
# tradebook_df

In [68]:
tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
tradebook_df['ROI%'] = tradebook_df['PnL'] * 100 / 10000000

In [69]:
tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year

In [70]:
tradebook_df['ROI%'].sum()

262.5046539868529

In [71]:
tradebook_df[tradebook_df['Trade Year'] >= 2024]

,Signal Time,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Points Captured,Remark,Qty,PnL,ROI%,Trade Year
801,2024-01-01 14:15:00,2024-01-01 15:15:00,48164.1500,48450.0000,2024-01-03 14:15:00,47718.9000,445.2500,RSI Oversold,1038.1165,462221.3825,4.6222,2024
802,2024-01-05 13:15:00,2024-01-05 14:15:00,47890.0500,48264.1000,2024-01-09 10:15:00,47899.2000,-9.1500,RSI Oversold,1044.0582,-9553.1326,-0.0955,2024
803,2024-01-09 14:15:00,2024-01-09 15:15:00,47206.2500,47877.0500,2024-01-10 13:15:00,47332.3000,-126.0500,RSI Oversold,1059.1818,-133509.8636,-1.3351,2024
804,2024-01-11 13:15:00,2024-01-11 14:15:00,47260.8500,47595.1500,2024-01-11 14:15:00,47432.5500,-171.7000,RSI Oversold,1057.9581,-181651.4091,-1.8165,2024
805,2024-01-17 10:15:00,2024-01-17 11:15:00,46586.5500,48138.8500,2024-01-19 09:15:00,46109.4500,477.1000,RSI Oversold,1073.2711,512057.6647,5.1206,2024
806,2024-01-19 10:15:00,2024-01-19 11:15:00,45895.6500,46249.8500,2024-01-19 11:15:00,45912.2500,-16.6000,RSI Oversold,1089.4279,-18084.5026,-0.1808,2024
807,2024-01-19 13:15:00,2024-01-19 14:15:00,45592.7000,46048.0500,2024-01-20 10:15:00,45951.9000,-359.2000,RSI Oversold,1096.6668,-393922.7113,-3.9392,2024
808,2024-01-23 10:15:00,2024-01-23 11:15:00,45645.3000,46580.3000,2024-01-24 10:15:00,45255.0000,390.3000,RSI Oversold,1095.4030,427535.8032,4.2754,2024
809,2024-01-24 11:15:00,2024-01-24 12:15:00,44664.8000,45485.5000,2024-01-24 14:15:00,45108.4500,-443.6500,RSI Oversold,1119.4498,-496643.8896,-4.9664,2024
810,2024-01-24 15:15:00,2024-01-25 09:15:00,45055.4500,45148.2000,2024-01-25 15:15:00,44988.5000,66.9500,RSI Oversold,1109.7437,74297.3381,0.7430,2024


In [72]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2018, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2018, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [73]:
variation = f'...{rsi_oversold}'
stats = generate_stats(tradebook_df, variation)

In [74]:
stats_dictionary = {}
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2018      32.1297           87  42.5287                4.1501             -2.4285     -22.2079       1.4468     ...50
2019      32.1468           85  41.1765                4.6074             -2.5823     -27.3435       1.1757     ...50
2020      72.1487           95  36.8421                8.5585             -3.7900     -57.1695       1.2620     ...50
2021      88.9018           85  37.6471                7.5834             -2.9013     -24.3032       3.6580     ...50
2022     -18.1526           93  27.9570                7.6056             -3.2223     -57.1091      -0.3179     ...50
2023      21.8525           80  27.5000                5.4604             -1.6944     -36.3518       0.6011     ...50
2024      -3.0387           88  38.6364                3.1314             -2.0279     -38.2086      -0.0795     ...50
Overall  225.9882          613  35.5456                5

In [75]:
stats_dictionary = {}

for ema_short_period in range(5, 16, 2):
    for ema_long_period in range(5, 24, 2):
        for rsi_period in range(2, 15, 2):
            for rsi_short_threshold in range(20, 81, 10):
                for rsi_oversold in range(20, 61, 10):
                    if (ema_short_period < ema_long_period) and (rsi_oversold <= rsi_short_threshold):
                        tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
                        tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
                        tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
                        tradebook_df['ROI%'] = tradebook_df['PnL'] * 100 / 10000000
                        tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year
                        variation = f'{ema_short_period}, {ema_long_period}, {rsi_period}, {rsi_short_threshold}, {rsi_oversold} -> {round(tradebook_df["ROI%"].sum())}%'
                        print(variation)
                        new_tb = tradebook_df
                        if len(new_tb) > 1:
                            new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                            stats = generate_stats(new_tb, variation)
                            for overall_roi_dd_ratio, stats_df in stats.items():
                                if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 9:
                                    print(stats_df.to_string())
                                    stats_dictionary[overall_roi_dd_ratio] = stats_df

5, 7, 2, 20, 20 -> 249%
5, 7, 2, 30, 20 -> 205%
5, 7, 2, 30, 30 -> 225%
5, 7, 2, 40, 20 -> 166%
5, 7, 2, 40, 30 -> 179%
5, 7, 2, 40, 40 -> 184%
5, 7, 2, 50, 20 -> 188%
5, 7, 2, 50, 30 -> 199%
5, 7, 2, 50, 40 -> 198%
5, 7, 2, 50, 50 -> 253%
5, 7, 2, 60, 20 -> 186%
5, 7, 2, 60, 30 -> 195%
5, 7, 2, 60, 40 -> 200%
5, 7, 2, 60, 50 -> 235%
5, 7, 2, 60, 60 -> 305%
5, 7, 2, 70, 20 -> 167%
5, 7, 2, 70, 30 -> 169%
5, 7, 2, 70, 40 -> 163%
5, 7, 2, 70, 50 -> 207%
5, 7, 2, 70, 60 -> 278%
5, 7, 2, 80, 20 -> 195%
5, 7, 2, 80, 30 -> 198%
5, 7, 2, 80, 40 -> 184%
5, 7, 2, 80, 50 -> 217%
5, 7, 2, 80, 60 -> 294%
5, 7, 4, 20, 20 -> 230%
5, 7, 4, 30, 20 -> 281%
5, 7, 4, 30, 30 -> 309%
5, 7, 4, 40, 20 -> 346%
5, 7, 4, 40, 30 -> 365%
5, 7, 4, 40, 40 -> 360%
5, 7, 4, 50, 20 -> 355%
5, 7, 4, 50, 30 -> 358%
5, 7, 4, 50, 40 -> 360%
5, 7, 4, 50, 50 -> 245%
5, 7, 4, 60, 20 -> 383%
5, 7, 4, 60, 30 -> 356%
5, 7, 4, 60, 40 -> 360%
5, 7, 4, 60, 50 -> 246%
5, 7, 4, 60, 60 -> 170%
5, 7, 4, 70, 20 -> 427%
5, 7, 4, 70, 30 

In [76]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

{}